In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import theano
import tensorflow
import keras
from keras.models import Sequential
from keras.layers import Dense
from sklearn.preprocessing import Imputer 
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout

WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.
Using TensorFlow backend.


In [3]:
dataset = pd.read_csv('F:\\Machine Learning\\Projects\\Titanic\\train.csv')
X_test = pd.read_csv('F:\\Machine Learning\\Projects\\Titanic\\test.csv')

In [4]:
# Proprocessing, building bigger classes and get rid of NaN values.

dataset_title = [i.split(",")[1].split(".")[0].strip() for i in dataset["Name"]]
dataset["Title"] = pd.Series(dataset_title)
dataset["Title"].value_counts()
dataset["Title"] = dataset["Title"].replace(['Lady', 'the Countess','Countess','Capt', 'Col','Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona', 'Ms', 'Mme', 'Mlle'], 'Rare')

dataset_title = [i.split(",")[1].split(".")[0].strip() for i in X_test["Name"]]
X_test["Title"] = pd.Series(dataset_title)
X_test["Title"].value_counts()
X_test["Title"] = X_test["Title"].replace(['Lady', 'the Countess','Countess','Capt', 'Col','Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona', 'Ms', 'Mme', 'Mlle'], 'Rare')

dataset["FamilyS"] = dataset["SibSp"] + dataset["Parch"] + 1
X_test["FamilyS"] = X_test["SibSp"] + X_test["Parch"] + 1



In [5]:
print dataset


     PassengerId  Survived  Pclass  \
0              1         0       3   
1              2         1       1   
2              3         1       3   
3              4         1       1   
4              5         0       3   
5              6         0       3   
6              7         0       1   
7              8         0       3   
8              9         1       3   
9             10         1       2   
10            11         1       3   
11            12         1       1   
12            13         0       3   
13            14         0       3   
14            15         0       3   
15            16         1       2   
16            17         0       3   
17            18         1       2   
18            19         0       3   
19            20         1       3   
20            21         0       2   
21            22         1       2   
22            23         1       3   
23            24         1       1   
24            25         0       3   
25          

In [47]:
def family(x):
    if x < 2:
        return "Single"
    elif x <= 2:
        return "Couple"
    elif x <= 4:
        return "InterM"
    else:
        return "Large"
dataset["FamilyS"] = dataset["FamilyS"].apply(family)
X_test["FamilyS"] = X_test["FamilyS"].apply(family)


In [6]:
dataset['Embarked'].fillna(dataset['Embarked'].mode()[0], inplace = True)
X_test['Embarked'].fillna(X_test['Embarked'].mode()[0], inplace = True)
dataset['Age'].fillna(dataset['Age'].median(), inplace = True)
X_test['Age'].fillna(X_test['Age'].median(), inplace = True)
X_test['Fare'].fillna(X_test['Fare'].median(), inplace = True)





In [7]:
dataset = dataset.drop(["PassengerId", "Cabin","Name", "SibSp", "Parch", "Ticket"], axis=1)
X_test = X_test.drop(["PassengerId", "Cabin","Name", "SibSp", "Parch", "Ticket"], axis=1)


In [8]:
X_train = dataset.iloc[:, 1:9].values
y_train = dataset.iloc[:, 0].values
X_test = X_test.values


In [9]:
## Male and Female are still labeled, so first they have to be labelled to numbers.
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelencoder_X_1 = LabelEncoder()
X_train[:, 1] = labelencoder_X_1.fit_transform(X_train[:, 1])
X_train[:, 4] = labelencoder_X_1.fit_transform(X_train[:, 4])
X_train[:, 5] = labelencoder_X_1.fit_transform(X_train[:, 5])
X_train[:, 6] = labelencoder_X_1.fit_transform(X_train[:, 6])

labelencoder_X_2 = LabelEncoder()
X_test[:, 1] = labelencoder_X_2.fit_transform(X_test[:, 1])
X_test[:, 4] = labelencoder_X_2.fit_transform(X_test[:, 4])
X_test[:, 5] = labelencoder_X_2.fit_transform(X_test[:, 5])
X_test[:, 6] = labelencoder_X_2.fit_transform(X_test[:, 6])


In [15]:
### Since Neural Networks can not handle categorical data, i will use dummy variables
onehotencoder = OneHotEncoder(categorical_features = [0, 1, 4, 5, 6])
X_train = onehotencoder.fit_transform(X_train).toarray()

onehotencoder = OneHotEncoder(categorical_features = [0, 1, 4, 5, 6])
X_test = onehotencoder.fit_transform(X_test).toarray()


In [18]:
def build_classifier():
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 19))
    classifier.add(Dropout(p = 0.2))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dropout(p = 0.2))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier
classifier = KerasClassifier(build_fn = build_classifier, batch_size = 25, epochs = 10)



In [ ]:
classifier.fit(X_train, y_train)



In [ ]:
# Predicting the Test set results, using the 0.5 threshold and get true or false values to build a confusion matrix.
y_pred = classifier.predict(X_test)
y_pred = pd.DataFrame(y_pred)


In [ ]:
### Converting the prediction to a 0/1 value at a 0.5 threshold.
def binary(x):
    if x > 0.5:
        return 1
    else:
        return 0

y_pred = y_pred[0].apply(binary)


In [ ]:
 #Since i deleted the dataset above, i load it again to get the PassengerId.

Ids = pd.read_csv('F:\\Machine Learning\\Projects\\Titanic\\test.csv')


In [ ]:
submission = pd.DataFrame({
        "PassengerId": Ids["PassengerId"],
        "Survived": y_pred
    })
    
submission.to_csv('titanic.csv', index=False)